# CNN With CIFAR 10

In [2]:
import torch
import torch.nn as nn

import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torchvision.utils
import torch.utils.data as Data

import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline